# Perk Comparison Analysis
This looks at a pair of decks after a common perk choice and breaksdown the changes in probabilities and attacks.

In [ ]:
import pandas as pd
from gloomhaven.deck import GloomhavenDeck
from gloomhaven.render import render_tables, format_table_to_hmtl

In [ ]:
deck1 = GloomhavenDeck()
deck2 = deck1.copy()

## Apply Perks

In [ ]:
# remove 2 "-1"s
assert deck1.remove_card("-1")
assert deck1.remove_card("-1")

# remove "-2" and add "0"
assert deck2.remove_card("-2")
assert deck2.add_card("0")

In [ ]:
from collections import Counter
import dictdiffer
diffs = dictdiffer.diff(
    Counter(deck1.card_list), Counter(deck2.card_list)
)
print("="*3, "changes", "="*3)
for diff in diffs:
    if diff[0] == "change":
        print(f"{diff[1]:>2}: {diff[2][0]:>2}  => {diff[2][1]:>2}")
    elif diff[0] == "remove":
        for d in diff[2]:
            print(f"{d[0]:>2}: {d[1]:>2}  => {0:>2}")
    elif diff[0] == "add":
        for d in diff[2]:
            print(f"{d[0]:>2}: {0:>2}  => {d[1]:>2}")

In [ ]:
attack_data = {}
base_attacks = [1, 2, 3, 4, 5]
samp_size = 100_000

def attacks(samp_size, attack):
    for _ in range(samp_size):
        yield attack

attack_data1, attack_data2 = {}, {}
for val in base_attacks:
    moves = deck1.simulate(attacks(samp_size, val))
    attack_data1[f"base_attack_{val}"] = [dmg for dmg, _ in moves]
    moves = deck2.simulate(attacks(samp_size, val))
    attack_data2[f"base_attack_{val}"] = [dmg for dmg, _ in moves]

attack_data1 = pd.DataFrame(attack_data1)
attack_data2 = pd.DataFrame(attack_data2)

In [ ]:
def get_counts(srs: pd.Series):
    srs = srs.value_counts()
    for attack_val in range(srs.index.max()+1):
        if attack_val not in srs.index:
            srs[attack_val] = 0
    srs = srs / samp_size
    srs.sort_index()
    return srs

In [ ]:
pdf_1 = attack_data1.apply(get_counts, axis=0)
pdf_2 = attack_data2.apply(get_counts, axis=0)

In [ ]:
def mode(x):
    return x.value_counts().index[0]

summ_1 = attack_data1.agg(["min", "median", "max", "mean", "std", mode])
summ_2 = attack_data2.agg(["min", "median", "max", "mean", "std", mode])

In [ ]:
def _color_red_or_green(val):
    if val < 0:
        return f"background-color: #f07067"
    elif val > 0:
        return f"background-color: #79ed85"

In [ ]:
diff_table = (summ_1.fillna(0).round(2) - summ_2.fillna(0).round(3))

In [ ]:
with open("../assets/perk_comparison.html", "w") as f:
    f.write(render_tables([
        (
            '"Remove -1x2" - "Remove -2, Add 0"',
            format_table_to_hmtl(diff_table, _color_red_or_green)
        ),
        (
            '"Remove -1x2" Summary Stats',
            format_table_to_hmtl(summ_1)
        ),
        (
           '"Remove -2, Add 0" Summary Stats',
            format_table_to_hmtl(summ_2) 
        )

    ]))